<a href="https://colab.research.google.com/github/datasistah/STAR_method/blob/main/StarMethodV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U git+https://github.com/lvwerra/trl.git git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/accelerate.git git+https://github.com/huggingface/peft.git
!pip install -q -U datasets bitsandbytes einops wandb
!pip install rouge-score


!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers


  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.8 MB/s et

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


https://vilsonrodrigues.medium.com/run-your-private-llm-falcon-7b-instruct-with-less-than-6gb-of-gpu-using-4-bit-quantization-ff1d4ffbabcc


#### Imports

In [ ]:
#CUDA error message
#!nvcc --version

In [ ]:
# !git clone https://github.com/TimDettmers/bitsandbytes.git
# %cd bitsandbytes
# !CUDA_VERSION=118 make cuda11x
# !python setup.py install


In [3]:
#CUDA error message
import os
os.environ["CUDA_VERSION"] = "118"
os.environ["LD_LIBRARY_PATH"] += ":/usr/local/cuda/lib64"


In [4]:
import torch, einops
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft.tuners.lora import LoraLayer

#### Load Quantized Model



In [5]:
#model_id = "ybelkada/falcon-7b-sharded-bf16"
model_id = "tiiuae/falcon-7b"
compute_dtype = getattr(torch, "float16")

#special config for bitsandbytes that enables 4bit quantization
bnb_config = BitsAndBytesConfig(
              load_in_4bit=True,
              bnb_4bit_quant_type="nf4",
              bnb_4bit_compute_dtype=compute_dtype,
              bnb_4bit_use_double_quant=True,
              )

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
                            model_id,
                            quantization_config=bnb_config,
                            trust_remote_code=True,
                            )


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    temperature=0.9,
    repetition_penalty=1.2,
    max_new_tokens=256,
    device_map="auto",
    do_sample=True
)



In [7]:
pipe("As a career coach, analyze the following interview response using the STAR method. If the method is used correctly, provide a summary of each component. If not, identify the missing components and suggest improvements. ")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[{'generated_text': 'As a career coach, analyze the following interview response using the STAR method. If the method is used correctly, provide a summary of each component. If not, identify the missing components and suggest improvements. [5 marks]\nMy ability to resolve any difficult employee issue led to our organization being recognized as one of the best places for new hires in the United States.\nLet’s recap what I have learned about your work history:\n- You have been working with the same organization for 13 years as an Administrative Assistant. It seems like you really love your job.\n- You have “the gift” that made you an excellent interviewer. Your ability to connect with people is top-notch!\n- You are an expert in organizing company data, and therefore your manager is very happy with your performance.\n- Your boss says that you excel in your job because you know how to balance many responsibilities at once.\n- You’re so great at what you do, that your company is going to p

In [8]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"


In [9]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 7.5 MB/s eta 0:00:00


In [10]:
#test PromptTemplate
# from langchain. prompts import PromptTemplate
# multi_prompt= PromptTemplate(input_variables=["adjective", "content"],
#                        template="suggest a good name for the {content} with the genre {adjective}",)

# print(multi_prompt.format(adjective="fictional", content="book"))

In [11]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
llm = HuggingFacePipeline(pipeline=pipe)

prompt = PromptTemplate(
      input_variables=['question', 'answer'],
      template="""
   As a career coach, analyze the following interview response using the STAR method. If the method is used correctly, provide a summary of each component. If not, identify the missing components and suggest improvements.
   question: {question}, answer: {answer}
   Return your response in json format.
    """
    )

question1 = "Tell me about a time when you received criticism that you thought was unfair."
answer1 = "In a previous role, my manager spoke to me about how long I was speaking to customers for on the telephone. I was trying to build up a positive rapport with our customers to help improve sales. My manager said I was taking too long. Although I felt the criticism was unfair because I was trying to help the company, it was my task to take on board his feedback to improve. I explained to my manager that I was trying to improve customer relations but that I fully understood there was a need to act with speed because other customers were waiting on the line that also needed my assistance. After reflecting on my performance, I made changes as per my manager's instructions and this improved the speed of my call handling rates, meaning I could help and assist more customers."
#test out prompt
print(prompt.format(question=question1, answer = answer1))



   As a career coach, analyze the following interview response using the STAR method. If the method is used correctly, provide a summary of each component. If not, identify the missing components and suggest improvements. 
   question: Tell me about a time when you received criticism that you thought was unfair., answer: In a previous role, my manager spoke to me about how long I was speaking to customers for on the telephone. I was trying to build up a positive rapport with our customers to help improve sales. My manager said I was taking too long. Although I felt the criticism was unfair because I was trying to help the company, it was my task to take on board his feedback to improve. I explained to my manager that I was trying to improve customer relations but that I fully understood there was a need to act with speed because other customers were waiting on the line that also needed my assistance. After reflecting on my performance, I made changes as per my manager's instructions a

In [12]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run(question=question1, answer = answer1)
response

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


'{\n     "name":"Mark",\n     "id":1,\n     "sal":40000,\n     "job": "career-coach",\n     "education": {\n        "school": "Utrecht University",\n        "degree": {\n            "title": "Phd in Law"\n         }\n     }\n}\n\n Hi. Have a look at the solution above. You need to make sure your object has a `job` field. This is the property you need to use to specify what class the object belongs to. Let me know how it goes. Thank you very much indeed for doing that. It worked great! That\'s great to hear. Good luck with the rest of the coding :) When you\'re done with the whole project, I\'m very interested in seeing how you\'ve done so far. I\'ll send you a message with my email address. Thanks again for the great comment :) Hello Sir, I have completed the Project..!! Does it looks good? Hi. It looks good, yes. Nice work. Sir...do you have any links to other interesting projects for me to look at? If you are interested in learning some more with'

#### Prepare Model for Training


*   activate gradient_checkpointing which significally reduces memory
*   print trainable parameters




In [13]:
from peft import prepare_model_for_kbit_training


model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [14]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

#### Create LoRa Config and Appply to Model


*   This is when we add the low ranking adapters/parameters to the model which are the only parameters that will be trained.

*   Increasing the "r" parameter in LoraConfig will add more adapters to the model.



In [15]:
 from peft import LoraConfig, get_peft_model

 peft_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=[
          "query_key_value",
         "dense",
         "dense_h_to_4h",
         "dense_4h_to_h"
        ]
    )

 model = get_peft_model(model, peft_config)
 print_trainable_parameters(model)

trainable params: 16318464 || all params: 3625063296 || trainable%: 0.4501566639679441


#### Prepare Data


> https://huggingface.co/docs/datasets/v1.1.3/loading_datasets.html



In [16]:
pwd

'/content'

In [17]:
!ls

drive  sample_data


In [52]:
import pandas as pd

df = pd.read_excel('/content/drive/MyDrive/StarMethod/Data/STAR_Method_Data.xlsx', sheet_name='Sheet2')
df.head()

,url,Question,Answer,STAR Components
0,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you received critici...,"In a previous role, my manager spoke to me abo...",Situation: My manager criticized the length of...
1,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you had to do someth...,I was working in a previous role as part of a ...,Situation: Team member's illness created a gap...
2,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you worked in a team.,"In a previous role, I was part of a team whose...",Situation: Team responsible for promoting a ne...
3,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you made a mistake.,"When I started working my last role, I was eag...",Situation: Overcommitted to tasks and missed a...
4,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you multitasked.,My manager asked for a volunteer to cover the ...,Situation: Volunteered to cover the work of a ...


In [53]:
#df['STAR Components'][188].replace("<br>", '')
df['output'] = df['STAR Components'].apply(lambda x: x.replace("<br>", ''))
df['output'][188]

'Missing Components: Task, Action, Result  Suggestion: Provide more context about the specific task, your actions, and the result.  Improved Answer: When I was working as an IT manager, we experienced a major system failure that halted our operations. My task was to identify the issue and restore the system as quickly as possible. I led my team in troubleshooting the problem, and we worked tirelessly to get the system back up. As a result, we were able to minimize the downtime and resume operations within a few hours.'

In [54]:
df['question'] = df['Question'].apply(lambda x: "question:" + x)
df['answer'] = df['Answer'].apply(lambda x: "answer:" + x)

df.head()

,url,Question,Answer,STAR Components,output,question,answer
0,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you received critici...,"In a previous role, my manager spoke to me abo...",Situation: My manager criticized the length of...,Situation: My manager criticized the length of...,question:Tell me about a time when you receive...,"answer:In a previous role, my manager spoke to..."
1,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you had to do someth...,I was working in a previous role as part of a ...,Situation: Team member's illness created a gap...,Situation: Team member's illness created a gap...,question:Tell me about a time when you had to ...,answer:I was working in a previous role as par...
2,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you worked in a team.,"In a previous role, I was part of a team whose...",Situation: Team responsible for promoting a ne...,Situation: Team responsible for promoting a ne...,question:Tell me about a time when you worked ...,"answer:In a previous role, I was part of a tea..."
3,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you made a mistake.,"When I started working my last role, I was eag...",Situation: Overcommitted to tasks and missed a...,Situation: Overcommitted to tasks and missed a...,question:Tell me about a time when you made a ...,"answer:When I started working my last role, I ..."
4,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you multitasked.,My manager asked for a volunteer to cover the ...,Situation: Volunteered to cover the work of a ...,Situation: Volunteered to cover the work of a ...,question:Tell me about a time when you multita...,answer:My manager asked for a volunteer to cov...


In [55]:
df["input"] = df[["question", "answer"]].apply(",".join, axis=1)
df['input'][186]

'question:Can you describe a time when you had to deal with a difficult customer?,answer:I dealt with a customer who was unhappy with our product.'

In [56]:
data= df[['input','output']]
data.head()

,input,output
0,question:Tell me about a time when you receive...,Situation: My manager criticized the length of...
1,question:Tell me about a time when you had to ...,Situation: Team member's illness created a gap...
2,question:Tell me about a time when you worked ...,Situation: Team responsible for promoting a ne...
3,question:Tell me about a time when you made a ...,Situation: Overcommitted to tasks and missed a...
4,question:Tell me about a time when you multita...,Situation: Volunteered to cover the work of a ...


In [57]:
data = data.sample(frac=1).reset_index(drop=True)
data.tail()

,input,output
186,question:Can you describe a time when you had ...,"Missing Components: Situation, Task, Action, R..."
187,question:Tell me about a time when you had to ...,"Missing Components: Situation, Task, Action, R..."
188,question:Can you describe a time when you had ...,"Missing Components: Situation, Task, Action, R..."
189,question:Tell me about a time when you disagre...,"Situation: As a Financial Analyst, I disagreed..."
190,question:Can you give me an example of a time ...,Situation: When working as a market researcher...


#### Example Before Fine Tuning

*   using data without training
*   convert text to pytorch tensors



In [36]:
pwd

'/content'

In [37]:
output_dir = "/drive/MyDrive/StarMethod/"

#### Training


*   TRL- HuggingFace "Transformer Reinforcement Learning" library
*   SFTTrainer- "supervised fine-tuning"
*   "paged_adamw_32bit" is powerful optimizer, helps to avoid memory overload



In [38]:
len(data)

191

In [ ]:
#data.to_json('/drive/MyDrive/StarMethod_training_data.jsonl', lines=True, orient='records')


In [39]:
type(data)

pandas.core.frame.DataFrame

In [40]:
from datasets import Dataset
temp_df = pd.DataFrame({"a": [1, 2, 3]})
temp_df

,a
0,1
1,2
2,3


In [41]:
temp_dataset = Dataset.from_pandas(temp_df)
temp_dataset

Dataset({
    features: ['a'],
    num_rows: 3
})

In [58]:
data.head()

,input,output
0,question:Can you tell me about a time when you...,"Missing Components: Situation, Task, Action, R..."
1,question:Tell me about a time when you deliver...,Situation: The product was out of stock. Task:...
2,"question:How do you handle criticism?,answer:I...","Missing Components: Situation, Task, Action, R..."
3,question:Can you tell me about a time when you...,"Missing Components: Situation, Task, Action, R..."
4,question:Describe a situation when you disagre...,Situation: Customer complaint about ordering d...


In [59]:
#convert pandas df with data to a Dataset format for training
dataset = Dataset.from_pandas(data)
dataset

Dataset({
    features: ['input', 'output'],
    num_rows: 191
})

In [60]:
dataset[0]

{'input': 'question:Can you tell me about a time when you had to use your analytical skills?,answer:I analyzed customer data to improve our marketing strategy.',
 'output': 'Missing Components: Situation, Task, Action, Result  Suggestion: Provide more details about the customer data, how you analyzed it, the changes you made to the marketing strategy, and the outcome.  Improved Answer: At my previous job, I noticed that our marketing strategy was not yielding the desired results. I analyzed our customer data and discovered that we were not effectively targeting the right demographic. Based on this analysis, I suggested changes to our marketing strategy to better target our key demographic. As a result, our customer engagement increased significantly.'}

In [61]:
#import supervised fine-tuning from HF library
#setup the trainer

from trl import SFTTrainer

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=30,
    optim="paged_adamw_32bit",
    save_steps=10,
    logging_steps=1,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=500,
    warmup_ratio=0.03
    )

In [74]:
#use following function when there are multiple features in Dataset

# def formatting_prompts_func(dataset):
#     output_texts = []
#     for i in range(len(dataset['input'])):
#         text = f"### input: {dataset['input'][i]}\n ### output: {dataset['output'][i]}"
#         output_texts.append(text)
#     return output_texts
def formatting_func(example):
    text = f"### input: {example['input']}\n ### output: {example['output']}"
    return text

In [75]:
#testing training data format
sample_data = dataset[:3]
formatting_func(sample_data)

"### input: ['question:Can you tell me about a time when you had to use your analytical skills?,answer:I analyzed customer data to improve our marketing strategy.', 'question:Tell me about a time when you delivered excellent customer service.,answer:In a previous role, I was speaking to a customer on the telephone. They wanted to buy a product but unfortunately, it was out of stock. They thanked me for my time and said they would look elsewhere. My task was to keep the customer on the call and find a workaround that resulted in them getting the product they needed. I asked them several questions about their needs and soon realized we had an alternative product that was more suited to their needs and considerably cheaper. After explaining the benefits of our alternative products, they agreed to purchase it. To thank them for being open to my alternative product suggestion, I gave them free shipping. They thanked me for my excellent customer service and left a five-star review online.', 

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    formatting_func=formatting_func,
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=True,
)
model.config.use_cache = False  # silence the warnings
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:268: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,1.852100
2,1.797400
3,2.009700
4,1.916300
5,1.933900
6,1.759700
7,1.887200
8,1.843700
9,1.718500
10,1.763800


/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:268: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:645: UserWarning: Length of IterableDataset <trl.trainer.utils.ConstantLengthDataset object at 0x7dbb58551e40> was reported to be 191 (when accessing len(dataloader)), but 192 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:645: UserWarning: Length of IterableDataset <trl.trainer.utils.ConstantLengthDataset object at 0x7dbb58551e40> was reported to be 191 (when accessing len(dataloader)), but 193 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:645: UserWarning: Length of IterableDataset <trl.trainer.utils.ConstantLengthDataset object at 0x7dbb

#### Save Model


*   local directory
*   HuggingFace



In [1]:
#Save to directory
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)

NameError: ignored

In [ ]:
# import locale
# def getpreferredencoding(do_setlocale = True):
#     return "UTF-8"
# locale.getpreferredencoding = getpreferredencoding

In [ ]:
!huggingface-cli login

In [ ]:
!git config --global credential.helper store

In [ ]:
#push model to HuggingFace Hub
model_to_save.push_to_hub(repo_id='datasistah/STARMethod_Interview_Model')

#### Import Model for Inference

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

config = PeftConfig.from_pretrained("datasistah/starMethod")
model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-7b", trust_remote_code=True)
model = PeftModel.from_pretrained(model, "datasistah/starMethod")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run(question=question1, answer = answer1)
response

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


'=> 2+3=?\n   Produce a cleaner output: The manager noticed my customer service skills were lacking. Upon reflection, I identified a way to improve my efficiency. Hence, from now on, I prioritized answering the phone quicker to facilitate better customer care. As a result, my average call handling time improved by 20% over the next month. <br> Improved Answer: In a former role, I learned the importance of prioritizing customers in my work. One day, a customer had an urgent query, but other customers were waiting for assistance. I acknowledged the fact that this was important, but I needed to ensure I provided excellent service to all customers. Reflecting on my performance, I determined a change in my approach would ensure I could prioritize customers waiting. This resulted in further improvements to my call handling times.'

In [ ]:
print(response)

=> 2+3=?
   Produce a cleaner output: The manager noticed my customer service skills were lacking. Upon reflection, I identified a way to improve my efficiency. Hence, from now on, I prioritized answering the phone quicker to facilitate better customer care. As a result, my average call handling time improved by 20% over the next month. <br> Improved Answer: In a former role, I learned the importance of prioritizing customers in my work. One day, a customer had an urgent query, but other customers were waiting for assistance. I acknowledged the fact that this was important, but I needed to ensure I provided excellent service to all customers. Reflecting on my performance, I determined a change in my approach would ensure I could prioritize customers waiting. This resulted in further improvements to my call handling times.


In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run(question="Tell me about a time when you received criticism that you thought was unfair.", answer = "In a previous role, my manager spoke to me about how long I was speaking to customers for on the telephone. I was trying to build up a positive rapport with our customers to help improve sales. My manager said I was taking too long. Although I felt the criticism was unfair because I was trying to help the company, it was my task to take on board his feedback to improve. I explained to my manager that I was trying to improve customer relations but that I fully understood there was a need to act with speed because other customers were waiting on the line that also needed my assistance. After reflecting on my performance, I made changes as per my manager's instructions and this improved the speed of my call handling rates, meaning I could help and assist more customers.")
response

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


"> Lorem Ipsum is used as a filler text for a form template. <br> Good Answer <br> Your answer should be at least one paragraph long. Where's the problem? What actions did you take? And how did you manage to solve the issue? Lorem Ipsum"

In [ ]:
question3 = "What is your greatest achievement?"
answer3 = "When I was a manager at Cafe X, we would always get a lot more customers than usual during the summer. It would almost always be packed, and sometimes, there were not enough servers to handle the work. So, I had to come up with a system so that we would not get overwhelmed during those 3+ months. To make sure we were prepared, I realized we needed a better schedule. For the next summer, I had a very strict shift schedule ready, and a reward system in case someone couldn’t show up for their shift. If someone called in sick, whoever filled in for them would be rewarded with extra pay. And if there was absolutely no one else available, I would personally fill in for them instead. Overall, the summer went pretty smoothly with minimal accidents. We didn’t get ANY negative reviews on TripAdvisor for the summer (whereas for the summer before that, we had several for slow service)."

llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run(question=question3, answer = answer3)
response


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


" Paste Code Snippet if you could improve performance by implementing new technologies or procedures. Describe how you handled the transition from the old technology to the new one. Did you set aside time for training employees? Was there any resistance? How did you deal with it?\n  +-+'' => Output:In my previous role as a Software Engineer, I was responsible for upgrading our company's outdated coding software. My task was to research different code generators and select the best option to suit our needs. After extensive screening, I identified a coding generator which was used by similar companies within our industry. The switchover took place smoothly and we experienced no disruption to our workflow. As an unexpected benefit, we were able to streamline our coding process, reducing the time spent writing code significantly. -> Solution A: Provide details about the situation, the actions taken, and the outcome. Commute: Start--> Task: Manage a team remotely.---------------------------